In [25]:
import pandas as pd

df = pd.read_pickle("202012-202103_processed.pkl")

In [2]:
df

,date,year,month,day,content_for_lda,content_for_sentiment,length,replyCount,retweetCount,likeCount,quoteCount,username,location
0,2020-12-01,2020,12,01,prior covid woman girl already expose differen...,"Prior to #COVID19, women and girls were alread...",244,1,10,10,1,UN_News_Centre,"Manhattan, NY"
1,2020-12-01,2020,12,01,hand washing key prevent spread respiratory in...,Hand washing is key in preventing the spread o...,161,0,0,0,0,WeweEwash,ABUJA
2,2020-12-01,2020,12,01,FromTheLeft want frighten people may see surge...,"#FromTheLeft: Don't want to frighten people, b...",130,0,1,1,0,AllSidesNow,United States
3,2020-12-01,2020,12,01,Ranger Delany Downieville c Tahoe National For...,Ranger Delany of Downieville c. 1945 Tahoe Nat...,155,0,0,1,0,PlacerMuseums,"Auburn, California"
4,2020-12-01,2020,12,01,spread rural ever since begin see large footho...,"""It's been spreading to rural, ever since the ...",192,0,0,0,0,Progress_NC,"North Carolina, USA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3691910,2021-03-31,2021,03,31,every American get high speed internet Part Ne...,Every American Is Getting High-Speed Internet ...,92,0,1,1,0,hardknoxfirst,"Columbus native, SE Ohio now."
3691911,2021-03-31,2021,03,31,Ontarios icu overwhelm record high occupancy d...,Ontario’s ICUs are being overwhelmed with reco...,238,7,31,122,1,StevenDelDuca,"Vaughan, Ontario"
3691912,2021-03-31,2021,03,31,Johnson Johnson COVID vaccine dose delay US fa...,Johnson &; Johnson #COVID19 vaccine doses are ...,78,0,0,0,0,paldhous,San Francisco
3691913,2021-03-31,2021,03,31,let world open let see right God Darwin COVID ...,Let the world open up and let's see who was ri...,109,0,0,0,0,Nearbas_,"Illinois, USA"


In [3]:
content=df.content_for_lda.to_list()

In [4]:
content=[x.lower() for x in content]

In [11]:
content

['prior covid woman girl already expose different form violence however pandemic heighten gender base violence human trafficking accord un office drugs crime lockeddownlockedin',
 'hand washing key prevent spread respiratory infection preventcovid stopthespread takeresponsibility covid tuesday tuesdaymotivation',
 'fromtheleft want frighten people may see surge upon surge fauci reporting fauci coronavirus covid',
 'ranger delany downieville c tahoe national forest collection shopping cybermonday check link covid history mondaymotivation',
 'spread rural ever since begin see large foothold rural affect place urbanized area ncpol covid',
 'stop search job covid motivationmonday mondaymotivation mcm',
 'report monday bring total number covid case report ciudad juarez death',
 'newmexico report new covid case additional death relate virus monday',
 'pm somewhere pm covid coronavirus coronaviruspandemic covidfirstresponders pmcheers time firstresponders stresponder thankyou clock tellingtim

In [12]:
remove_list = ['-pron-','not','say']
corpus = []
for index in range(len(content)):
    temp = content[index].split(" ")
    #temp = [x.lower() for x in temp]
    sen = [word for word in temp if not (len(word) < 3 or word in remove_list)] #去掉小于3个字符或在remove_list中的词
    corpus.append(sen)

In [13]:
len(corpus)

3691915

In [14]:
del content

In [15]:
import numpy as np
import time
from gensim import corpora

t1=time.time()
dictionary = corpora.Dictionary(corpus,prune_at=3000000)
t2=time.time()
print('生成词典耗时',t2-t1,'s')

C:\ProgramData\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [16]:
dictionary.filter_extremes(no_below=75,no_above=0.9)  # 删掉只在不超过75个文本中出现过的词，删掉在90%及以上的文本都出现了的词
#dictionary.filter_n_most_frequent(10)
dictionary.compactify()  # 去掉因删除词汇而出现的空白

In [17]:
print(dictionary)

Dictionary(25940 unique tokens: ['accord', 'already', 'base', 'crime', 'different']...)


In [19]:
t1=time.time()
vec = [dictionary.doc2bow(s) for s in corpus]
t2=time.time()
print('生成词袋模型耗时',t2-t1,'s')

In [20]:
del corpus

In [22]:
vec[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 2),
 (18, 1)]

In [21]:
vec[3482316]

[]

In [26]:
import logging

if len(vec) == len(df):
    print('文档数量正常')
else:
    logging.critical('需检查文档索引')

文档数量正常


In [38]:
dictionary.save('dict.dict')

2021-04-20 17:26:47,804 : INFO : Dictionary lifecycle event {'fname_or_handle': 'dict.dict', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2021-04-20T17:26:47.804120', 'gensim': '4.0.1', 'python': '3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.18362-SP0', 'event': 'saving'}
2021-04-20 17:26:47,841 : INFO : saved dict.dict


In [39]:
corpora.MmCorpus.serialize('bow.mm',vec)

2021-04-20 17:28:19,893 : INFO : storing corpus in Matrix Market format to bow.mm
2021-04-20 17:28:19,896 : INFO : saving sparse matrix to bow.mm
2021-04-20 17:28:19,897 : INFO : PROGRESS: saving document #0
2021-04-20 17:28:19,925 : INFO : PROGRESS: saving document #1000
2021-04-20 17:28:19,948 : INFO : PROGRESS: saving document #2000
2021-04-20 17:28:19,968 : INFO : PROGRESS: saving document #3000
2021-04-20 17:28:19,991 : INFO : PROGRESS: saving document #4000
2021-04-20 17:28:20,017 : INFO : PROGRESS: saving document #5000
2021-04-20 17:28:20,042 : INFO : PROGRESS: saving document #6000
2021-04-20 17:28:20,064 : INFO : PROGRESS: saving document #7000
2021-04-20 17:28:20,088 : INFO : PROGRESS: saving document #8000
2021-04-20 17:28:20,111 : INFO : PROGRESS: saving document #9000
2021-04-20 17:28:20,133 : INFO : PROGRESS: saving document #10000
2021-04-20 17:28:20,156 : INFO : PROGRESS: saving document #11000
2021-04-20 17:28:20,179 : INFO : PROGRESS: saving document #12000
2021-04-2

2021-04-20 17:28:22,547 : INFO : PROGRESS: saving document #122000
2021-04-20 17:28:22,565 : INFO : PROGRESS: saving document #123000
2021-04-20 17:28:22,583 : INFO : PROGRESS: saving document #124000
2021-04-20 17:28:22,603 : INFO : PROGRESS: saving document #125000
2021-04-20 17:28:22,622 : INFO : PROGRESS: saving document #126000
2021-04-20 17:28:22,643 : INFO : PROGRESS: saving document #127000
2021-04-20 17:28:22,661 : INFO : PROGRESS: saving document #128000
2021-04-20 17:28:22,682 : INFO : PROGRESS: saving document #129000
2021-04-20 17:28:22,701 : INFO : PROGRESS: saving document #130000
2021-04-20 17:28:22,722 : INFO : PROGRESS: saving document #131000
2021-04-20 17:28:22,742 : INFO : PROGRESS: saving document #132000
2021-04-20 17:28:22,761 : INFO : PROGRESS: saving document #133000
2021-04-20 17:28:22,781 : INFO : PROGRESS: saving document #134000
2021-04-20 17:28:22,800 : INFO : PROGRESS: saving document #135000
2021-04-20 17:28:22,819 : INFO : PROGRESS: saving document #13

2021-04-20 17:28:25,049 : INFO : PROGRESS: saving document #245000
2021-04-20 17:28:25,069 : INFO : PROGRESS: saving document #246000
2021-04-20 17:28:25,092 : INFO : PROGRESS: saving document #247000
2021-04-20 17:28:25,114 : INFO : PROGRESS: saving document #248000
2021-04-20 17:28:25,134 : INFO : PROGRESS: saving document #249000
2021-04-20 17:28:25,157 : INFO : PROGRESS: saving document #250000
2021-04-20 17:28:25,178 : INFO : PROGRESS: saving document #251000
2021-04-20 17:28:25,198 : INFO : PROGRESS: saving document #252000
2021-04-20 17:28:25,218 : INFO : PROGRESS: saving document #253000
2021-04-20 17:28:25,241 : INFO : PROGRESS: saving document #254000
2021-04-20 17:28:25,261 : INFO : PROGRESS: saving document #255000
2021-04-20 17:28:25,281 : INFO : PROGRESS: saving document #256000
2021-04-20 17:28:25,301 : INFO : PROGRESS: saving document #257000
2021-04-20 17:28:25,321 : INFO : PROGRESS: saving document #258000
2021-04-20 17:28:25,341 : INFO : PROGRESS: saving document #25

2021-04-20 17:28:27,662 : INFO : PROGRESS: saving document #368000
2021-04-20 17:28:27,696 : INFO : PROGRESS: saving document #369000
2021-04-20 17:28:27,714 : INFO : PROGRESS: saving document #370000
2021-04-20 17:28:27,734 : INFO : PROGRESS: saving document #371000
2021-04-20 17:28:27,754 : INFO : PROGRESS: saving document #372000
2021-04-20 17:28:27,773 : INFO : PROGRESS: saving document #373000
2021-04-20 17:28:27,793 : INFO : PROGRESS: saving document #374000
2021-04-20 17:28:27,813 : INFO : PROGRESS: saving document #375000
2021-04-20 17:28:27,833 : INFO : PROGRESS: saving document #376000
2021-04-20 17:28:27,852 : INFO : PROGRESS: saving document #377000
2021-04-20 17:28:27,873 : INFO : PROGRESS: saving document #378000
2021-04-20 17:28:27,896 : INFO : PROGRESS: saving document #379000
2021-04-20 17:28:27,924 : INFO : PROGRESS: saving document #380000
2021-04-20 17:28:27,941 : INFO : PROGRESS: saving document #381000
2021-04-20 17:28:27,960 : INFO : PROGRESS: saving document #38

2021-04-20 17:28:30,185 : INFO : PROGRESS: saving document #491000
2021-04-20 17:28:30,210 : INFO : PROGRESS: saving document #492000
2021-04-20 17:28:30,231 : INFO : PROGRESS: saving document #493000
2021-04-20 17:28:30,252 : INFO : PROGRESS: saving document #494000
2021-04-20 17:28:30,274 : INFO : PROGRESS: saving document #495000
2021-04-20 17:28:30,294 : INFO : PROGRESS: saving document #496000
2021-04-20 17:28:30,319 : INFO : PROGRESS: saving document #497000
2021-04-20 17:28:30,342 : INFO : PROGRESS: saving document #498000
2021-04-20 17:28:30,364 : INFO : PROGRESS: saving document #499000
2021-04-20 17:28:30,383 : INFO : PROGRESS: saving document #500000
2021-04-20 17:28:30,404 : INFO : PROGRESS: saving document #501000
2021-04-20 17:28:30,423 : INFO : PROGRESS: saving document #502000
2021-04-20 17:28:30,442 : INFO : PROGRESS: saving document #503000
2021-04-20 17:28:30,460 : INFO : PROGRESS: saving document #504000
2021-04-20 17:28:30,479 : INFO : PROGRESS: saving document #50

...

2021-04-20 17:29:28,347 : INFO : PROGRESS: saving document #3161000
2021-04-20 17:29:28,372 : INFO : PROGRESS: saving document #3162000
2021-04-20 17:29:28,397 : INFO : PROGRESS: saving document #3163000
2021-04-20 17:29:28,426 : INFO : PROGRESS: saving document #3164000
2021-04-20 17:29:28,454 : INFO : PROGRESS: saving document #3165000
2021-04-20 17:29:28,480 : INFO : PROGRESS: saving document #3166000
2021-04-20 17:29:28,506 : INFO : PROGRESS: saving document #3167000
2021-04-20 17:29:28,532 : INFO : PROGRESS: saving document #3168000
2021-04-20 17:29:28,556 : INFO : PROGRESS: saving document #3169000
2021-04-20 17:29:28,582 : INFO : PROGRESS: saving document #3170000
2021-04-20 17:29:28,609 : INFO : PROGRESS: saving document #3171000
2021-04-20 17:29:28,632 : INFO : PROGRESS: saving document #3172000
2021-04-20 17:29:28,656 : INFO : PROGRESS: saving document #3173000
2021-04-20 17:29:28,680 : INFO : PROGRESS: saving document #3174000
2021-04-20 17:29:28,703 : INFO : PROGRESS: savin

2021-04-20 17:29:31,341 : INFO : PROGRESS: saving document #3282000
2021-04-20 17:29:31,364 : INFO : PROGRESS: saving document #3283000
2021-04-20 17:29:31,387 : INFO : PROGRESS: saving document #3284000
2021-04-20 17:29:31,422 : INFO : PROGRESS: saving document #3285000
2021-04-20 17:29:31,446 : INFO : PROGRESS: saving document #3286000
2021-04-20 17:29:31,473 : INFO : PROGRESS: saving document #3287000
2021-04-20 17:29:31,496 : INFO : PROGRESS: saving document #3288000
2021-04-20 17:29:31,522 : INFO : PROGRESS: saving document #3289000
2021-04-20 17:29:31,547 : INFO : PROGRESS: saving document #3290000
2021-04-20 17:29:31,572 : INFO : PROGRESS: saving document #3291000
2021-04-20 17:29:31,594 : INFO : PROGRESS: saving document #3292000
2021-04-20 17:29:31,618 : INFO : PROGRESS: saving document #3293000
2021-04-20 17:29:31,642 : INFO : PROGRESS: saving document #3294000
2021-04-20 17:29:31,669 : INFO : PROGRESS: saving document #3295000
2021-04-20 17:29:31,693 : INFO : PROGRESS: savin

2021-04-20 17:29:34,325 : INFO : PROGRESS: saving document #3403000
2021-04-20 17:29:34,347 : INFO : PROGRESS: saving document #3404000
2021-04-20 17:29:34,370 : INFO : PROGRESS: saving document #3405000
2021-04-20 17:29:34,392 : INFO : PROGRESS: saving document #3406000
2021-04-20 17:29:34,413 : INFO : PROGRESS: saving document #3407000
2021-04-20 17:29:34,436 : INFO : PROGRESS: saving document #3408000
2021-04-20 17:29:34,457 : INFO : PROGRESS: saving document #3409000
2021-04-20 17:29:34,481 : INFO : PROGRESS: saving document #3410000
2021-04-20 17:29:34,504 : INFO : PROGRESS: saving document #3411000
2021-04-20 17:29:34,526 : INFO : PROGRESS: saving document #3412000
2021-04-20 17:29:34,549 : INFO : PROGRESS: saving document #3413000
2021-04-20 17:29:34,576 : INFO : PROGRESS: saving document #3414000
2021-04-20 17:29:34,597 : INFO : PROGRESS: saving document #3415000
2021-04-20 17:29:34,620 : INFO : PROGRESS: saving document #3416000
2021-04-20 17:29:34,642 : INFO : PROGRESS: savin

2021-04-20 17:29:37,064 : INFO : PROGRESS: saving document #3524000
2021-04-20 17:29:37,089 : INFO : PROGRESS: saving document #3525000
2021-04-20 17:29:37,111 : INFO : PROGRESS: saving document #3526000
2021-04-20 17:29:37,133 : INFO : PROGRESS: saving document #3527000
2021-04-20 17:29:37,155 : INFO : PROGRESS: saving document #3528000
2021-04-20 17:29:37,176 : INFO : PROGRESS: saving document #3529000
2021-04-20 17:29:37,199 : INFO : PROGRESS: saving document #3530000
2021-04-20 17:29:37,238 : INFO : PROGRESS: saving document #3531000
2021-04-20 17:29:37,261 : INFO : PROGRESS: saving document #3532000
2021-04-20 17:29:37,284 : INFO : PROGRESS: saving document #3533000
2021-04-20 17:29:37,308 : INFO : PROGRESS: saving document #3534000
2021-04-20 17:29:37,332 : INFO : PROGRESS: saving document #3535000
2021-04-20 17:29:37,358 : INFO : PROGRESS: saving document #3536000
2021-04-20 17:29:37,386 : INFO : PROGRESS: saving document #3537000
2021-04-20 17:29:37,408 : INFO : PROGRESS: savin

2021-04-20 17:29:39,854 : INFO : PROGRESS: saving document #3645000
2021-04-20 17:29:39,877 : INFO : PROGRESS: saving document #3646000
2021-04-20 17:29:39,899 : INFO : PROGRESS: saving document #3647000
2021-04-20 17:29:39,922 : INFO : PROGRESS: saving document #3648000
2021-04-20 17:29:39,946 : INFO : PROGRESS: saving document #3649000
2021-04-20 17:29:39,969 : INFO : PROGRESS: saving document #3650000
2021-04-20 17:29:39,991 : INFO : PROGRESS: saving document #3651000
2021-04-20 17:29:40,011 : INFO : PROGRESS: saving document #3652000
2021-04-20 17:29:40,033 : INFO : PROGRESS: saving document #3653000
2021-04-20 17:29:40,055 : INFO : PROGRESS: saving document #3654000
2021-04-20 17:29:40,077 : INFO : PROGRESS: saving document #3655000
2021-04-20 17:29:40,100 : INFO : PROGRESS: saving document #3656000
2021-04-20 17:29:40,121 : INFO : PROGRESS: saving document #3657000
2021-04-20 17:29:40,144 : INFO : PROGRESS: saving document #3658000
2021-04-20 17:29:40,167 : INFO : PROGRESS: savin